本版本对应于yunxuan-v7

在Shunted-Transformer的基础上修改为高光谱版本

特征提取器修改为3D-Conv-->>多尺度2D-Conv【kernel_size=[1,3,5]】-->>高斯加权映射

其中kernel_size=1的特征未采用卷积方式而是直接对查询像素进行加权映射

In [1]:
!pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 KB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 23.1 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from functools import partial

from timm.models.layers import DropPath, to_2tuple, trunc_normal_
from timm.models.registry import register_model
from timm.models.vision_transformer import _cfg
import math

In [3]:
class Mlp(nn.Module):
  def __init__(self,in_features,hidden_features=None,out_features=None,act_layer=nn.GELU,drop=0.):
    super().__init__()
    out_features = out_features or in_features
    hidden_feature = hidden_features or in_features
    self.fc1 = nn.Linear(in_features,hidden_features)
    self.dwconv = DWConv(hidden_features)
    self.act = act_layer()
    self.fc2 = nn.Linear(hidden_features, out_features)
    self.drop = nn.Dropout(drop)
    self.apply(self._init_weights)
  def _init_weights(self,m):
    if isinstance(m,nn.Linear):
      trunc_normal_(m.weight,std=.02)
      if isinstance(m, nn.Linear) and m.bias is not None:
        nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.LayerNorm):
      nn.init.constant_(m.bias, 0)
      nn.init.constant_(m.weight, 1.0)
    elif isinstance(m, nn.Conv2d):
      fan_out = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
      fan_out //= m.groups
      m.weight.data.normal_(0, math.sqrt(2.0 / fan_out))
      if m.bias is not None:
        m.bias.data.zero_()
  def forward(self,x,H,W):
    print("******<<Mlp start>>******")
    print("input data x.shape:",x.shape)
    x = self.fc1(x)
    x = self.act(x + self.dwconv(x, H, W))
    x = self.drop(x)
    x = self.fc2(x)
    x = self.drop(x)
    print("output data x.shape:",x.shape)
    print("******<<Mlp end>>******")
    return x

In [4]:
class Attention(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0., sr_ratio=1):
        super().__init__()
        assert dim % num_heads == 0, f"dim {dim} should be divided by num_heads {num_heads}."

        self.dim = dim
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5

        self.q = nn.Linear(dim, dim, bias=qkv_bias)

        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)


        self.sr_ratio = sr_ratio
        if sr_ratio > 1:
            self.act = nn.GELU()
            if sr_ratio==8:
                self.sr1 = nn.Conv2d(dim, dim, kernel_size=8, stride=8)
                self.norm1 = nn.LayerNorm(dim)
                self.sr2 = nn.Conv2d(dim, dim, kernel_size=4, stride=4)
                self.norm2 = nn.LayerNorm(dim)
            if sr_ratio==4:
                self.sr1 = nn.Conv2d(dim, dim, kernel_size=4, stride=4)
                self.norm1 = nn.LayerNorm(dim)
                self.sr2 = nn.Conv2d(dim, dim, kernel_size=2, stride=2)
                self.norm2 = nn.LayerNorm(dim)
            if sr_ratio==2:
                self.sr1 = nn.Conv2d(dim, dim, kernel_size=2, stride=2)
                self.norm1 = nn.LayerNorm(dim)
                self.sr2 = nn.Conv2d(dim, dim, kernel_size=1, stride=1)
                self.norm2 = nn.LayerNorm(dim)
            self.kv1 = nn.Linear(dim, dim, bias=qkv_bias)
            self.kv2 = nn.Linear(dim, dim, bias=qkv_bias)
            self.local_conv1 = nn.Conv2d(dim//2, dim//2, kernel_size=3, padding=1, stride=1, groups=dim//2)
            self.local_conv2 = nn.Conv2d(dim//2, dim//2, kernel_size=3, padding=1, stride=1, groups=dim//2)
        else:
            self.kv = nn.Linear(dim, dim * 2, bias=qkv_bias)
            self.local_conv = nn.Conv2d(dim, dim, kernel_size=3, padding=1, stride=1, groups=dim)
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)
        elif isinstance(m, nn.Conv2d):
            fan_out = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            fan_out //= m.groups
            m.weight.data.normal_(0, math.sqrt(2.0 / fan_out))
            if m.bias is not None:
                m.bias.data.zero_()

    def forward(self, x, H, W):
        print("******<<attention start>>******")
        print("input data x.shape:",x.shape)
        print("sr_ratio:",self.sr_ratio)
        B, N, C = x.shape
        print("B:{} | N:{} | C:{}".format(int(B),int(N),int(C)))
        q = self.q(x).reshape(B, N, self.num_heads, C // self.num_heads).permute(0, 2, 1, 3)
        print("q.shape:",q.shape)
        if self.sr_ratio > 1:
                x_ = x.permute(0, 2, 1).reshape(B, C, H, W)
                print("after reshape x.shape:",x_.shape)
                x_1 = self.act(self.norm1(self.sr1(x_).reshape(B, C, -1).permute(0, 2, 1)))#
                print("x_1.shape:",x_1.shape)
                x_2 = self.act(self.norm2(self.sr2(x_).reshape(B, C, -1).permute(0, 2, 1)))
                print("x_2.shape:",x_2.shape)
                kv1 = self.kv1(x_1).reshape(B, -1, 2, self.num_heads//2, C // self.num_heads).permute(2, 0, 3, 1, 4)
                kv2 = self.kv2(x_2).reshape(B, -1, 2, self.num_heads//2, C // self.num_heads).permute(2, 0, 3, 1, 4)
                k1, v1 = kv1[0], kv1[1] #B head N C
                k2, v2 = kv2[0], kv2[1]
                print("k1.shape:{} | v1.shape:{} | k2.shape:{} | v2.shape:{}".format(str(k1.shape),str(v1.shape),str(k2.shape),str(v2.shape),))
                attn1 = (q[:, :self.num_heads//2] @ k1.transpose(-2, -1)) * self.scale
                print("after attn1.shape:",attn1.shape)
                attn1 = attn1.softmax(dim=-1)
                attn1 = self.attn_drop(attn1)
                print("after softmax&dropout attn1.shape:",attn1.shape)
                v1 = v1 + self.local_conv1(v1.transpose(1, 2).reshape(B, -1, C//2).
                                        transpose(1, 2).view(B,C//2, H//self.sr_ratio, W//self.sr_ratio)).\
                    view(B, C//2, -1).view(B, self.num_heads//2, C // self.num_heads, -1).transpose(-1, -2)
                print("after argument v1.shape:",v1.shape)
                x1 = (attn1 @ v1).transpose(1, 2).reshape(B, N, C//2)
                print("self-attention x1.shape:",x1.shape)
                attn2 = (q[:, self.num_heads // 2:] @ k2.transpose(-2, -1)) * self.scale
                print("after softmax&dropout attn2.shape:",attn2.shape)
                attn2 = attn2.softmax(dim=-1)
                attn2 = self.attn_drop(attn2)
                print("after softmax&dropout attn2.shape:",attn2.shape)
                v2 = v2 + self.local_conv2(v2.transpose(1, 2).reshape(B, -1, C//2).
                                        transpose(1, 2).view(B, C//2, H*2//self.sr_ratio, W*2//self.sr_ratio)).\
                    view(B, C//2, -1).view(B, self.num_heads//2, C // self.num_heads, -1).transpose(-1, -2)
                print("after argument v2.shape:",v2.shape)
                x2 = (attn2 @ v2).transpose(1, 2).reshape(B, N, C//2)
                print("self-attention x2.shape:",x2.shape)

                x = torch.cat([x1,x2], dim=-1)
                print("after self-attention cat x.shape:",x.shape)
        else:
            kv = self.kv(x).reshape(B, -1, 2, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
            k, v = kv[0], kv[1]
            print("k.shape:{} | v.shape:{}".format(str(k.shape),str(v.shape)))

            attn = (q @ k.transpose(-2, -1)) * self.scale
            attn = attn.softmax(dim=-1)
            attn = self.attn_drop(attn)
            print("after softmax&dropout attn.shape:",attn.shape)

            x = (attn @ v).transpose(1, 2).reshape(B, N, C) + self.local_conv(v.transpose(1, 2).reshape(B, N, C).
                                        transpose(1, 2).view(B,C, H, W)).view(B, C, N).transpose(1, 2)
            print("self-attention x.shape:",x.shape)
        x = self.proj(x)
        print("after Linear x.shape:",x.shape)
        x = self.proj_drop(x)
        print("after dropout x.shape:",x.shape)
        print("******<<attention end>>******")

        return x

In [5]:
#每个block包括一个attention和一个前馈网络
class Block(nn.Module):

    def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=False, qk_scale=None, drop=0., attn_drop=0.,
                 drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm, sr_ratio=1):
        super().__init__()
        self.norm1 = norm_layer(dim)
        self.attn = Attention(
            dim,
            num_heads=num_heads, qkv_bias=qkv_bias, qk_scale=qk_scale,
            attn_drop=attn_drop, proj_drop=drop, sr_ratio=sr_ratio)
        # NOTE: drop path for stochastic depth, we shall see if this is better than dropout here
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)

        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)
        elif isinstance(m, nn.Conv2d):
            fan_out = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            fan_out //= m.groups
            m.weight.data.normal_(0, math.sqrt(2.0 / fan_out))
            if m.bias is not None:
                m.bias.data.zero_()

    def forward(self, x, H, W):
        x = x + self.drop_path(self.attn(self.norm1(x), H, W))
        x = x + self.drop_path(self.mlp(self.norm2(x), H, W))

        return x

In [6]:
class OverlapPatchEmbed(nn.Module):
    """ Image to Patch Embedding
    """

    def __init__(self, img_size=224, patch_size=7, stride=4, in_chans=3, embed_dim=768):
        super().__init__()
        img_size = to_2tuple(img_size)
        patch_size = to_2tuple(patch_size)

        self.img_size = img_size
        self.patch_size = patch_size
        self.H, self.W = img_size[0] // patch_size[0], img_size[1] // patch_size[1]
        self.num_patches = self.H * self.W
        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=stride,
                              padding=(patch_size[0] // 2, patch_size[1] // 2))
        self.norm = nn.LayerNorm(embed_dim)

        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)
        elif isinstance(m, nn.Conv2d):
            fan_out = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            fan_out //= m.groups
            m.weight.data.normal_(0, math.sqrt(2.0 / fan_out))
            if m.bias is not None:
                m.bias.data.zero_()

    def forward(self, x):
        print("******<<OverlapPatchEmbed start>>******")
        print("input data x.shape:",x.shape)
        x = self.proj(x)
        print("after proj x.shape:",x.shape)
        _, _, H, W = x.shape
        x = x.flatten(2).transpose(1, 2)
        print("after flatten&transpose x.shape:",x.shape)
        x = self.norm(x)
        print("after norm x.shape:",x.shape)
        print("output data x.shape:",x.shape)
        print("******<<OverlapPatchEmbed end>>******")

        return x, H, W

In [7]:
#此函数用于网络初始阶段生成token
class Head(nn.Module):
    def __init__(self, num):
        super(Head, self).__init__()
        stem = [nn.Conv2d(3, 64, 7, 2, padding=3, bias=False), nn.BatchNorm2d(64), nn.ReLU(True)]
        for i in range(num):
            stem.append(nn.Conv2d(64, 64, 3, 1, padding=1, bias=False))
            stem.append(nn.BatchNorm2d(64))
            stem.append(nn.ReLU(True))
            #print("循环内部的卷积未改变特征图大小")
        stem.append(nn.Conv2d(64, 64, kernel_size=2, stride=2))
        self.conv = nn.Sequential(*stem)
        self.norm = nn.LayerNorm(64)
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)
        elif isinstance(m, nn.Conv2d):
            fan_out = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            fan_out //= m.groups
            m.weight.data.normal_(0, math.sqrt(2.0 / fan_out))
            if m.bias is not None:
                m.bias.data.zero_()
    def forward(self, x):
        print("******<<Head start>>******")
        print("input data x.shape:",x.shape)
        x = self.conv(x)
        print("after conv x.shape:",x.shape)
        _, _, H, W = x.shape
        x = x.flatten(2).transpose(1, 2)
        print("after flatten&transpose x.shape:",x.shape)
        x = self.norm(x)
        print("after norm x.shape:",x.shape)
        print("H:{}  |  W:{}".format(int(H),int(W)))
        print("output data x.shape:",x.shape)
        print("******<<Head end>>******")
        return x, H,W

In [8]:
!pip install einops

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 KB 2.4 MB/s eta 0:00:00


In [9]:
from einops import rearrange

In [10]:
#此函数用于网络初始阶段生成token
class Head_High(nn.Module):
    def __init__(self, num):
        super(Head_High, self).__init__()
        self.conv3d_features = nn.Sequential(
            nn.Conv3d(1, out_channels=8, kernel_size=(3, 3, 3)),
            nn.BatchNorm3d(8),
            nn.ReLU(),
        )
        self.conv2d_features = nn.Sequential(
            nn.Conv2d(in_channels=8*28, out_channels=64, kernel_size=(3, 3)),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )

        self.conv2d_features_2 = nn.Sequential(
            nn.Conv2d(in_channels=8*28, out_channels=64, kernel_size=(5, 5)),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        #self.conv = nn.Sequential(*stem)

        #高斯加权模块
        #................................................
        self.token_wA = nn.Parameter(torch.empty(1,140,64),requires_grad=True)# Tokenization parameters (1,4,64)
        torch.nn.init.xavier_normal_(self.token_wA)
        self.token_wV = nn.Parameter(torch.empty(1,64,64),requires_grad=True)# Tokenization parameters
        torch.nn.init.xavier_normal_(self.token_wV)

        self.token_wA_1 = nn.Parameter(torch.empty(1,140,64),requires_grad=True)# Tokenization parameters (1,4,64)
        torch.nn.init.xavier_normal_(self.token_wA_1)
        self.token_wV_1 = nn.Parameter(torch.empty(1,64,64),requires_grad=True)# Tokenization parameters
        torch.nn.init.xavier_normal_(self.token_wV_1)

        self.token_wA_2 = nn.Parameter(torch.empty(1,4,30),requires_grad=True)# Tokenization parameters (1,4,64)
        torch.nn.init.xavier_normal_(self.token_wA_2)
        self.token_wV_2 = nn.Parameter(torch.empty(1,1,64),requires_grad=True)# Tokenization parameters
        torch.nn.init.xavier_normal_(self.token_wV_2)

        #................................................
        #位置编码模块
        #................................................
        self.pos_embedding = nn.Parameter(torch.empty(1,144,64))
        torch.nn.init.normal_(self.pos_embedding, std=.02)
        #................................................
        #cls_token模块
        #................................................
        #self.cls_token = nn.Parameter(torch.zeros(1, 1, 64))
        #self.to_cls_token = nn.Identity()
        #................................................

        self.dropout_T = nn.Dropout(0.1)


        self.norm = nn.LayerNorm(64)
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)
        elif isinstance(m, nn.Conv2d):
            fan_out = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            fan_out //= m.groups
            m.weight.data.normal_(0, math.sqrt(2.0 / fan_out))
            if m.bias is not None:
                m.bias.data.zero_()
    def forward(self, x):
        x_spec = x[:,:,:,10:11,10:11]
        print("******<<Head start>>******")
        print("input data x.shape:",x.shape)
        print("input data x_spec.shape:",x_spec.shape)
        x = self.conv3d_features(x)
        print("after conv3d x.shape:",x.shape)
        x = rearrange(x, 'b c h w y -> b (c h) w y')
        print("after rearrange x.shape:",x.shape)
        #多尺度特征
        #...................................................
        x_3d = x
        x = self.conv2d_features(x)
        print("after conv2d x.shape:",x.shape)

        x_m2 = self.conv2d_features_2(x_3d)
        print("x_m2.shape:",x_m2.shape)
        #...................................................

        _, _, H, W = x.shape
        x = x.flatten(2).transpose(1, 2)
        print("after flatten&transpose x.shape:",x.shape)
        x_m2 = x_m2.flatten(2).transpose(1, 2)
        print("after flatten&transpose x_m2.shape:",x_m2.shape)

        #高斯加权模块1
        #................................................
        print("高斯加权模块1")
        wa = rearrange(self.token_wA, 'b h w -> b w h')#生成weight matrix wa(1,64,81)
        print("weight matrix wa.shape:",wa.shape)
        A = torch.einsum('bij,bjk->bik', x, wa)#利用weight matrix 和 x 生成Sematic group(1,289,81)
        print("Sematic group A.shape:",A.shape)
        A = rearrange(A, 'b h w -> b w h')  # 转置Sematic group以进行SOFTMAX  (1,81,289)
        print("after transpose Sematic group A.shape:",A.shape)
        A = A.softmax(dim=-1)
        print("after softmax Sematic group A.shape:",A.shape)
        VV = torch.einsum('bij,bjk->bik', x, self.token_wV)# x(1,289,64) wV(1,64,64) VV(1,289,64)
        print("weight matrix2 wV.shape:",self.token_wV.shape)
        print("weighted x VV.shape:",VV.shape)#(1,81,64),与x一样；就相当于给x赋了一层权重
        T = torch.einsum('bij,bjk->bik', A, VV)
        print("the data token T.shape:",T.shape)
        #................................................
        #高斯加权模块2
        #................................................
        print("高斯加权模块2")
        wa1 = rearrange(self.token_wA_1, 'b h w -> b w h')#生成weight matrix wa(1,64,81)
        print("weight matrix wa1.shape:",wa1.shape)
        A1 = torch.einsum('bij,bjk->bik', x_m2, wa1)#利用weight matrix 和 x 生成Sematic group(1,289,81)
        print("Sematic group A1.shape:",A1.shape)
        A1 = rearrange(A1, 'b h w -> b w h')  # 转置Sematic group以进行SOFTMAX  (1,81,289)
        print("after transpose Sematic group A1.shape:",A1.shape)
        A1 = A1.softmax(dim=-1)
        print("after softmax Sematic group A1.shape:",A1.shape)
        VV1 = torch.einsum('bij,bjk->bik', x_m2, self.token_wV_1)# x(1,289,64) wV(1,64,64) VV(1,289,64)
        print("weight matrix2 wV1.shape:",self.token_wV_1.shape)
        print("weighted x_m2 VV1.shape:",VV1.shape)#(1,81,64),与x一样；就相当于给x赋了一层权重
        T1 = torch.einsum('bij,bjk->bik', A1, VV1)
        print("the data token T1.shape:",T1.shape)

        #高斯加权模块3
        #................................................
        print("高斯加权模块3")
        x_spec = rearrange(x_spec,'b c h w y -> b (c h) (w y)')
        x_spec = x_spec.transpose(-1,-2)
        print("after rearrange x_spec.shape:",x_spec.shape)
        wa2 = rearrange(self.token_wA_2, 'b h w -> b w h')
        print("weight matrix wa1.shape:",wa2.shape)
        A2 = torch.einsum('bij,bjk->bik', x_spec, wa2)#利用weight matrix 和 x 生成Sematic group(1,289,81)
        print("Sematic group A2.shape:",A2.shape)
        A2 = rearrange(A2, 'b h w -> b w h')  # 转置Sematic group以进行SOFTMAX  (1,81,289)
        print("after transpose Sematic group A2.shape:",A2.shape)
        A2 = A2.softmax(dim=-1)
        print("after softmax Sematic group A2.shape:",A2.shape)
        T2 = torch.einsum('bij,bjk->bik', A2, self.token_wV_2)# x(1,289,64) wV(1,64,64) VV(1,289,64)
        print("weight matrix2 wV2.shape:",self.token_wV_2.shape)       
        print("the data token T2.shape:",T2.shape)
        print("高斯加权结束！！！")
        #................................................
        #多尺度特征融合
        T = T+T1
        print("初步融合特征T.shape:",T.shape)
        T = torch.cat((T, T2), dim=1)
        print("融合后的特此尺度T.shape:",T.shape)

        #................................................

        #位置编码模块
        #...............................................
        T += self.pos_embedding
        print("after add the pos_information T.shape:",T.shape)
        #...............................................
        T = self.dropout_T(T)

        x = self.norm(T)
        print("after norm x.shape:",x.shape)
        H=W=int((x.shape[1])**(0.5))
        print("H:{}  |  W:{}".format(int(H),int(W)))
        print("output data x.shape:",x.shape)
        print("******<<Head end>>******")
        return x, H, W

In [11]:
class ShuntedTransformer(nn.Module):
    def __init__(self, img_size=224, patch_size=16, in_chans=3, num_classes=16, embed_dims=[64, 128, 256, 512],
                 num_heads=[1, 2, 4, 8], mlp_ratios=[4, 4, 4, 4], qkv_bias=False, qk_scale=None, drop_rate=0.,
                 attn_drop_rate=0., drop_path_rate=0., norm_layer=nn.LayerNorm,
                 depths=[3, 4, 6, 3], sr_ratios=[8, 4, 2, 1], num_stages=4, num_conv=0):
        super().__init__()
        self.num_classes = num_classes
        self.depths = depths
        self.num_stages = num_stages

        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, sum(depths))]  # stochastic depth decay rule
        cur = 0

        for i in range(num_stages):
            if i ==0:
                patch_embed = Head_High(num_conv)#
            else:
                patch_embed = OverlapPatchEmbed(img_size=img_size if i == 0 else img_size // (2 ** (i + 1)),
                                            patch_size=7 if i == 0 else 3,
                                            stride=4 if i == 0 else 2,
                                            in_chans=in_chans if i == 0 else embed_dims[i - 1],
                                            embed_dim=embed_dims[i])

            block = nn.ModuleList([Block(
                dim=embed_dims[i], num_heads=num_heads[i], mlp_ratio=mlp_ratios[i], qkv_bias=qkv_bias, qk_scale=qk_scale,
                drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[cur + j], norm_layer=norm_layer,
                sr_ratio=sr_ratios[i])
                for j in range(depths[i])])#depth表示第i个stage中block的数量
            norm = norm_layer(embed_dims[i])#embed_dims表示每个stage的channel大小
            cur += depths[i]

            setattr(self, f"patch_embed{i + 1}", patch_embed)
            setattr(self, f"block{i + 1}", block)
            setattr(self, f"norm{i + 1}", norm)

        # classification head
        self.head = nn.Linear(embed_dims[3], num_classes) if num_classes > 0 else nn.Identity()

        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)
        elif isinstance(m, nn.Conv2d):
            fan_out = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            fan_out //= m.groups
            m.weight.data.normal_(0, math.sqrt(2.0 / fan_out))
            if m.bias is not None:
                m.bias.data.zero_()

    def freeze_patch_emb(self):
        self.patch_embed1.requires_grad = False

    @torch.jit.ignore
    def no_weight_decay(self):
        return {'pos_embed1', 'pos_embed2', 'pos_embed3', 'pos_embed4', 'cls_token'}  # has pos_embed may be better

    def get_classifier(self):
        return self.head

    def reset_classifier(self, num_classes, global_pool=''):
        self.num_classes = num_classes
        self.head = nn.Linear(self.embed_dim, num_classes) if num_classes > 0 else nn.Identity()

    def forward_features(self, x):
        print("the input data of net x.shape:",x.shape)
        B = x.shape[0]
        #分阶段处理
        for i in range(self.num_stages):
            print("---------<<num_stages:{}satrt>>---------".format(int(i)))
            print("input data x.shape:",x.shape)
            patch_embed = getattr(self, f"patch_embed{i + 1}")
            block = getattr(self, f"block{i + 1}")
            norm = getattr(self, f"norm{i + 1}")
            x, H, W = patch_embed(x)
            #print("after patch_embed x.shape:",x.shape)
            print("H:{}  |  W:{}".format(int(H),int(W)))
            for blk in block:
                x = blk(x, H, W)
            x = norm(x)
            print("after bolck finish x.shape:",x.shape)
            if i != self.num_stages - 1:
                x = x.reshape(B, H, W, -1).permute(0, 3, 1, 2).contiguous()
            print("output data x.shape:",x.shape)
            print("---------<<num_stages:{}end>>---------".format(int(i)))
        return x.mean(dim=1)#压缩第二个维度

    def forward(self, x):
        x = self.forward_features(x)
        print("after forward_feature x.shape:",x.shape)
        print("|||||||||||||<<forward_feature finish>>|||||||||||||")
        x = self.head(x)
        print("after classi_head x.shape:",x.shape)
        print("|||||||||||||<<classi_head finish>>|||||||||||||")

        return x

In [12]:
class DWConv(nn.Module):
    def __init__(self, dim=768):
        super(DWConv, self).__init__()
        self.dwconv = nn.Conv2d(dim, dim, 3, 1, 1, bias=True, groups=dim)

    def forward(self, x, H, W):
        B, N, C = x.shape
        x = x.transpose(1, 2).view(B, C, H, W)
        x = self.dwconv(x)
        x = x.flatten(2).transpose(1, 2)

        return x

In [13]:
def _conv_filter(state_dict, patch_size=16):
    """ convert patch embedding weight from manual patchify + linear proj to conv"""
    out_dict = {}
    for k, v in state_dict.items():
        if 'patch_embed.proj.weight' in k:
            v = v.reshape((v.shape[0], 3, patch_size, patch_size))
        out_dict[k] = v

    return out_dict

In [14]:
@register_model
def shunted_t(pretrained=False, **kwargs):
    model = ShuntedTransformer(
        patch_size=4, embed_dims=[64, 128, 256, 512], num_heads=[2, 4, 8, 16], mlp_ratios=[8, 8, 4, 4], qkv_bias=True,
        norm_layer=partial(nn.LayerNorm, eps=1e-6), depths=[1, 2, 4, 1], sr_ratios=[8, 4, 2, 1], num_conv=0,
        **kwargs)
    model.default_cfg = _cfg()

    return model

In [15]:
@register_model
def shunted_b(pretrained=False, **kwargs):
    model = ShuntedTransformer(
        patch_size=4, embed_dims=[64, 128, 256, 512], num_heads=[2, 4, 8, 16], mlp_ratios=[8, 8, 4, 4], qkv_bias=True,
        norm_layer=partial(nn.LayerNorm, eps=1e-6), depths=[3, 4, 24, 2], sr_ratios=[8, 4, 2, 1], num_conv=2,
        **kwargs)
    model.default_cfg = _cfg()

    return model

In [16]:
if __name__ == '__main__':
    model = shunted_t()
    model.eval()
    #print(model)
    input = torch.randn(8,1,30,21,21)
    y = model(input)
    print(y.size())

the input data of net x.shape: torch.Size([8, 1, 30, 21, 21])
---------<<num_stages:0satrt>>---------
input data x.shape: torch.Size([8, 1, 30, 21, 21])
******<<Head start>>******
input data x.shape: torch.Size([8, 1, 30, 21, 21])
input data x_spec.shape: torch.Size([8, 1, 30, 1, 1])
after conv3d x.shape: torch.Size([8, 8, 28, 19, 19])
after rearrange x.shape: torch.Size([8, 224, 19, 19])
after conv2d x.shape: torch.Size([8, 64, 17, 17])
x_m2.shape: torch.Size([8, 64, 15, 15])
after flatten&transpose x.shape: torch.Size([8, 289, 64])
after flatten&transpose x_m2.shape: torch.Size([8, 225, 64])
高斯加权模块1
weight matrix wa.shape: torch.Size([1, 64, 140])
Sematic group A.shape: torch.Size([8, 289, 140])
after transpose Sematic group A.shape: torch.Size([8, 140, 289])
after softmax Sematic group A.shape: torch.Size([8, 140, 289])
weight matrix2 wV.shape: torch.Size([1, 64, 64])
weighted x VV.shape: torch.Size([8, 289, 64])
the data token T.shape: torch.Size([8, 140, 64])
高斯加权模块2
weight matrix